## **Import Rquired Libraries**

In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv
import sys
import warnings
from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore
from qdrant_client.http.models import Filter, FieldCondition, MatchValue
from qdrant_client.http.models import VectorParams,Distance,Filter,MatchValue,FieldCondition

In [2]:
## Ignore warnings for cleaner output
warnings.filterwarnings("ignore")

## **Install Dependencies**

In [3]:
!{sys.executable} -m pip install -r ../requirements.txt


[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## **Setup Working Directory**

In [5]:
## Add project root to system path
CURRENT_DIR=Path.cwd()
PROJECT_ROOT=CURRENT_DIR

while not (PROJECT_ROOT/'src').exists() and PROJECT_ROOT!=PROJECT_ROOT.parent:
    PROJECT_ROOT=PROJECT_ROOT.parent 

if not (PROJECT_ROOT/'src').exists():
    raise Exception("Could not find the 'src' folder.Check the folder structure")

## add project root to the python path
sys.path.insert(0,str(PROJECT_ROOT))

from src.context_engineering.config import(
        EMBEDDING_MODEL,
        PROVIDER,
        CHUNKS_DIR,
        LLM_MODEL,
        TOP_K,
        LLM_MODEL,
        EXPANDED_K,
        get_api_keys,
        load_faq
)

openrouter_api_key=get_api_keys(provider="openrouter",env_path=PROJECT_ROOT/".env")
openai_api_key=get_api_keys(provider="open-ai",env_path=PROJECT_ROOT/".env")

## check whether API keys there or not
if not  openrouter_api_key or not openai_api_key:
    raise ValueError(
        "API keys are not Found.\n"
        "Please add those API keys into the .env file if there is anything missing."
    )

print("="*60)
print("Enviroment Loadeding............")
print("="*60)

print(f"Provider:{PROVIDER}")
print(f"Project Root Directory:{PROJECT_ROOT}")
print("✅Environment Loaded")

Enviroment Loadeding............
Provider:openrouter
Project Root Directory:c:\UOC pdf\AI Engineering Bootcamp\mini-project-03
✅Environment Loaded


## **Import CHAT Services(RAG,CAG AND CRAG)**

In [6]:
## import user define modules
from src.context_engineering.chat_services.rag_service import QdrantRAGService
from src.context_engineering.chat_services.cag_service import QdrantCAGService
from src.context_engineering.chat_services.crag_service import QdrantCRAGService


print("="*60)
print("Loading CHAT Services............")
print("Available Services:")
print("- Qdrant RAG Service")
print("- Qdrant CAG Service")
print("- Qdrant CCAG Service")


Loading CHAT Services............
Available Services:
- Qdrant RAG Service
- Qdrant CAG Service
- Qdrant CCAG Service


## **Build Qdrant Vector DB Index**

#### **Loading Embedding and Chat Model**

In [7]:
## import user define modules
from src.context_engineering.services.embedding import _get_embedding_model
from src.context_engineering.services.response_llm import get_chat_llm

chat_llm=get_chat_llm(path=PROJECT_ROOT/".env")
embedding_model=_get_embedding_model(env_path=PROJECT_ROOT/".env")

print(f"✅ LLM initialized: {LLM_MODEL}")
print(f"✅ Embeddings initialized: {EMBEDDING_MODEL}")
print(f"🌐 Provider: {PROVIDER}")


✅ LLM initialized: openai/gpt-4o-mini
✅ Embeddings initialized: openai/text-embedding-3-large
🌐 Provider: openrouter


#### **Initialize the Qdrant Vector DB**

In [8]:
## import qdrant configuration paramters(URL and API key)
qdrant_configurations=get_api_keys(provider="qdrant",env_path=PROJECT_ROOT/".env")

## initialize the qdrant client
client=QdrantClient(
    url=qdrant_configurations["url"],
    api_key=qdrant_configurations["key"],
    timeout=60
)

## create a qdrant collection 
collection_name = "Pime-Land-Real-Estate-AI-Assistant"

vectorstore=QdrantVectorStore(
    client=client,
    collection_name=collection_name,
    embedding=embedding_model,
    content_payload_key="text",
    metadata_payload_key="metadata"
)

# Convert to retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={
         "k": TOP_K,
         "filter":Filter(
             must=[
                 FieldCondition(
                     key="metadata.chunking_strategy",
                     match=MatchValue(value="Sliding Strategy")
                 )
                  
             ]
         )
        }
    )

## **1️⃣ RAG with Service**

In [9]:
## Initialize RAG service
rag_service=QdrantRAGService(
    retriever=retriever,
    llm=chat_llm,
    k=TOP_K
)

print("✅ RAG Service initialized: QdrantRAGService")
print("="*60)
print(f"Retrieved {TOP_K} relevant chunks for the query.")

✅ RAG Service initialized: QdrantRAGService
Retrieved 5 relevant chunks for the query.


In [10]:
# Generate Answer with RAG Service
USER_QUERY = "Apartments in Kurunagala"

print(f"🔍 Query: {USER_QUERY}\n")
print("=" * 80)
print("GENERATING ANSWER WITH RAG SERVICE...")
print("=" * 80)

result = rag_service.generate_response(USER_QUERY)

print(f"\n⏱️  Generation time: {result['generation_time']:.2f}s")
print(f"📄 Documents used: {result['num_docs']}")
print("\n" + "=" * 80)
print("ANSWER")
print("=" * 80)
print(result['answer'])
print("\n" + "=" * 80)
print("EVIDENCE URLS")
print("=" * 80)
for url in result['evidence_urls']:
    print(f"  - {url}")

🔍 Query: Apartments in Kurunagala

GENERATING ANSWER WITH RAG SERVICE...

⏱️  Generation time: 7.19s
📄 Documents used: 5

ANSWER
1. **Key Details**
   - Property Name: Apartments
   - Price: 86,000,000 LKR
   - Bedrooms: 2-3
   - Bathrooms: 2-3
   - Source: [Prime Lands Apartments](https://www.primelands.lk/apartment/complete/en)

2. **Answer**
   The available apartments in Kurunegala are priced at 86,000,000 LKR. They feature 2 to 3 bedrooms and 2 to 3 bathrooms. Unfortunately, no additional amenities are specified for these apartments. For more detailed information, you can visit the official Prime Lands website [here](https://www.primelands.lk/apartment/complete/en).

3. **Next Steps**
   For confirmation of pricing, availability, or to schedule a site visit, please contact Prime Lands directly. You can reach them through their hotline or by filling out the inquiry form on their website.

EVIDENCE URLS
  - https://www.primelands.lk/land/KURUNEGALA-WEHERA/sin
  - https://www.primela

## **2️⃣ CAG With Semantic Matching**

In [12]:
## create a new FAQs qdrant collection 
collection_name_cache = "Pime-Land-Real-Estate-AI-Assistant-FQA-Cache"

if client.collection_exists(collection_name_cache):
    print(f"Collection '{collection_name_cache}' already exists in Qdrant.")
    print("Deleting the existing collection to create a new one with the same name....")
    client.delete_collection(collection_name=collection_name_cache)

    print(f"Creating a new collection '{collection_name_cache}' with the same name....")
    client.create_collection(
        collection_name=collection_name_cache,
        vectors_config=VectorParams(
            size=3072,
            distance=Distance.COSINE    
        )
    )
else:
    print(f"Collection '{collection_name_cache}' does not exist. Creating a new collection....")
    client.create_collection(
        collection_name=collection_name_cache,
        vectors_config=VectorParams(
            size=3072,
            distance=Distance.COSINE
        )
    )

print(f"✅  FAQs Collection '{collection_name_cache}' is ready in Qdrant.")

Collection 'Pime-Land-Real-Estate-AI-Assistant-FQA-Cache' already exists in Qdrant.
Deleting the existing collection to create a new one with the same name....
Creating a new collection 'Pime-Land-Real-Estate-AI-Assistant-FQA-Cache' with the same name....
✅  FAQs Collection 'Pime-Land-Real-Estate-AI-Assistant-FQA-Cache' is ready in Qdrant.


In [15]:
## Initialize CAG service
cag_service=QdrantCAGService(
    rag_service=rag_service,
    client=client,
    cache_collection=collection_name_cache,
    embedding_model=embedding_model
)
print("✅ CAG Service initialized: QdrantCAGService")
print("="*60)

✅ CAG Service initialized: QdrantCAGService


In [16]:
## load FAQ yaml file
count_faq=0

print("="*60)
print("Loading FAQs file.................")
faq_file=load_faq(PROJECT_ROOT/'config'/'FAQ.yaml')

for collection,question in faq_file.items():
    for q in question:
        count_faq+=1

print(f"📋 Found {count_faq} FAQs in config/FAQs.yaml")
print("\n📝 Sample FAQs:")
for collection,question in faq_file.items():
    print(f"   - {question[0]}")

print(f"   ... and {count_faq - 8} more\n")
print("FAQs File Loaded")

Loading FAQs file.................
📋 Found 53 FAQs in config/FAQs.yaml

📝 Sample FAQs:
   - What is Prime Lands?
   - What projects does Prime Lands currently offer?
   - What is the price per perch?
   - How do I reserve a land plot?
   - Are Prime Lands properties government approved?
   - Do you support bank loans?
   - Do your projects have electricity and water?
   - Is buying land from Prime Lands a good investment?
   ... and 45 more

FAQs File Loaded


In [17]:
cag_service.warm_faqs(faq_quetions=faq_file)

Generating answer for FAQ:What is Prime Lands?
Generating answer for FAQ:Tell me about Prime Lands real estate.
Generating answer for FAQ:Is Prime Lands a registered company?
Generating answer for FAQ:Where is Prime Lands located?
Generating answer for FAQ:What is the office address of Prime Lands?
Generating answer for FAQ:How can I contact Prime Lands?
Generating answer for FAQ:What is the phone number of Prime Lands?
Generating answer for FAQ:What are the working hours of Prime Lands?
Generating answer for FAQ:What projects does Prime Lands currently offer?
Generating answer for FAQ:Where are Prime Lands projects located?
Generating answer for FAQ:Do you have land for sale in Negombo?
Generating answer for FAQ:Do you have land for sale near Colombo?
Generating answer for FAQ:Which districts do you operate in?
Generating answer for FAQ:Are your lands close to main roads?
Generating answer for FAQ:Are there residential projects available?
Generating answer for FAQ:Do you offer commerc

In [18]:
# Generate Answer with CAG Service
USER_QUERY = "Apartments in Kurunagala"

print(f"🔍 Query: {USER_QUERY}\n")
print("=" * 80)
result =cag_service.generate_response(USER_QUERY)

if result["source"]=="cache":
    print(f"\n⏱️  Generation time: {result['generation_time']:.2f}s")
    print("\n" + "=" * 80)
    print("ANSWER")
    print("=" * 80)
    print(result['answer'])
    print("\n" + "=" * 80)
    print("Source")
    print(result["source"])


else:
    print(f"\n⏱️  Generation time: {result['answer']['generation_time']:.2f}s")
    print(f"📄 Documents used: {result['answer']['num_docs']}")
    print("\n" + "=" * 80)
    print("ANSWER")
    print("=" * 80)
    print(result['answer']['answer'])
    print("\n" + "=" * 80)
    print("EVIDENCE URLS")
    print("=" * 80)
    for url in result["answer"]['evidence_urls']:
        print(f"  - {url}")

🔍 Query: Apartments in Kurunagala

❌ Cache Miss → Generating answer using RAG...

⏱️  Generation time: 6.09s
📄 Documents used: 5

ANSWER
1. **Key Details**
   - Property Name: Apartments
   - Price: 86,000,000 LKR
   - Bedrooms: 2-3
   - Bathrooms: 2-3
   - Source: [Prime Lands Apartments](https://www.primelands.lk/apartment/complete/en)

2. **Answer**
   The available apartments in Kurunegala are priced at 86,000,000 LKR. They feature 2 to 3 bedrooms and 2 to 3 bathrooms. Unfortunately, no additional amenities are specified for these apartments. For more detailed information, you can visit the official Prime Lands website [here](https://www.primelands.lk/apartment/complete/en).

3. **Next Steps**
   For confirmation of pricing, availability, or to schedule a site visit, please contact Prime Lands directly. You can reach them through their hotline or by filling out the inquiry form on their website.

EVIDENCE URLS
  - https://www.primelands.lk/land/KURUNEGALA-WEHERA/sin
  - https://www

## **3️⃣CRAG With Retrievel Improvement**

In [21]:
crag_service=QdrantCRAGService(
    retriever=retriever,
    llm=chat_llm,
    expanded_k=EXPANDED_K,
    initial_k=TOP_K
    
)

print("✅ CRAG Service initialized: QdrantCRAGService")
print(f"🎯 Initial retrieval: top-{TOP_K}")
print(f"🔄 Corrective retrieval: top-{EXPANDED_K}")

✅ CRAG Service initialized: QdrantCRAGService
🎯 Initial retrieval: top-5
🔄 Corrective retrieval: top-8


In [22]:
# Generate Answer with CRAG Service
USER_QUERY="I’m planning to invest in a residential land project from Prime Lands within the next 6 months. " \
"Can you compare two ongoing projects in Colombo suburbs in terms of price per perch, payment plans, legal approvals, " \
"expected ROI, and infrastructure development nearby? Also, explain the risks involved"

print(f"🔍 Query: {USER_QUERY}\n")
print("=" * 80)
print("GENERATING ANSWER WITH CRAG SERVICE...")
print("=" * 80)

result = crag_service.generate_crag_response(USER_QUERY)

print(f"\n⏱️  Generation time: {result['generation_time']:.2f}s")
print(f"📄 Documents used: {result['docs_used']}")
print("\n" + "=" * 80)
print("ANSWER")
print("=" * 80)
print(result['answer'])
print("\n" + "=" * 80)
print("EVIDENCE URLS")
print("=" * 80)
for url in result['evidence_urls']:
    print(f"  - {url}")

🔍 Query: I’m planning to invest in a residential land project from Prime Lands within the next 6 months. Can you compare two ongoing projects in Colombo suburbs in terms of price per perch, payment plans, legal approvals, expected ROI, and infrastructure development nearby? Also, explain the risks involved

GENERATING ANSWER WITH CRAG SERVICE...
🔍 Query: I’m planning to invest in a residential land project from Prime Lands within the next 6 months. Can you compare two ongoing projects in Colombo suburbs in terms of price per perch, payment plans, legal approvals, expected ROI, and infrastructure development nearby? Also, explain the risks involved
🎯 Confidence threshold: 0.6

1️⃣  Initial retrieval (k=5)...
   📊 Confidence: 0.37
   ⚠️  Low confidence - applying corrective retrieval...

2️⃣  Corrective retrieval (k=8, expanded)...
   📊 Corrected confidence: 0.32
   📈 Confidence improved by -4.3%

3️⃣  Generating answer...

⏱️  Generation time: 5.40s
📄 Documents used: 8

ANSWER
1. **Key 

## **Performance Evaluation**

#### **CAG cache Effectiveness**

In [38]:
## load queries.yaml file
count_queries=0

print("="*60)
print("Simulating Queries.................")
query_file=load_faq(PROJECT_ROOT/'config'/'queries.yaml')

for collection,question in query_file.items():
    for q in question:
        count_queries+=1

print(f"📋 Simulated {count_queries} Queries in config/queries.yaml")
print("\n📝 Sample Queries:")
for collection,question in query_file.items():
    print(f"   - {question[0]}")

print(f"   ... and {count_queries - 8} more\n")
print("Queries Loaded")

Simulating Queries.................
📋 Simulated 100 Queries in config/queries.yaml

📝 Sample Queries:
   - What is Prime Lands?
   - What projects does Prime Lands currently have in Colombo?
   - What is the price per perch?
   - Compare two ongoing Prime Lands projects in Colombo suburbs in terms of price, location advantages, and legal approvals.
   - What amenities are included in gated projects?
   - Are Prime Lands projects approved by local authorities?
   - What financing options are available when purchasing land from Prime Lands?
   ... and 92 more

Queries Loaded


In [39]:
## Initilize the cache evalution
from src.context_engineering.performance_evaluation.cache_effectiveness import CAGEvaluator
cache_evalution=CAGEvaluator(
    cag_service=cag_service,
    cost_per_llm_call=0.0012 ## cost per LLM call
)

In [40]:
## covnert all queries to list format
def flatten_queries(query_dict: dict) -> list:
        """
        Convert categorized query dictionary into a flat list.

        Args:
            query_dict (dict): YAML-loaded query dictionary.

        Returns:
            list: Combined list of all queries.
    """
        all_queries = []
        for category, queries in query_dict.items():
            all_queries.extend(queries)
        return all_queries
    
query_list=flatten_queries(query_file)

In [ ]:
print(f"🔄 CAG cache effectiveness starting....................")
print("="*80)
performance_result=cache_evalution.evaluate(queries=query_list,simulate_n=100)
print("="*80)
print(f"✅ Finished cache checking of {count_queries} queries")

🔄 CAG cache effectiveness starting....................
❌ Cache Miss → Generating answer using RAG...
❌ Cache Miss → Generating answer using RAG...
❌ Cache Miss → Generating answer using RAG...
❌ Cache Miss → Generating answer using RAG...
❌ Cache Miss → Generating answer using RAG...
❌ Cache Miss → Generating answer using RAG...
❌ Cache Miss → Generating answer using RAG...
❌ Cache Miss → Generating answer using RAG...
✅ Cache Hit
❌ Cache Miss → Generating answer using RAG...
❌ Cache Miss → Generating answer using RAG...
❌ Cache Miss → Generating answer using RAG...
❌ Cache Miss → Generating answer using RAG...
❌ Cache Miss → Generating answer using RAG...
❌ Cache Miss → Generating answer using RAG...
❌ Cache Miss → Generating answer using RAG...
❌ Cache Miss → Generating answer using RAG...
❌ Cache Miss → Generating answer using RAG...
❌ Cache Miss → Generating answer using RAG...
❌ Cache Miss → Generating answer using RAG...
❌ Cache Miss → Generating answer using RAG...
❌ Cache Miss 

In [67]:
performance_result

{'Total Queries': 100,
 'Cache Hits': 15,
 'Cache Hit Rate': 0.15,
 'Avg Total Latency (s)': 7.429,
 'Avg Cache Latency (s)': 1.258,
 'Avg RAG Latency (s)': 8.518,
 'Latency Improvement (%)': 85.24,
 'Estimated Cost Without Cache ($)': 0.12,
 'Estimated Cost With Cache ($)': 0.102,
 'Estimated Cost Saved ($)': 0.018}

#### **CRAG Correction Impact**

In [ ]:
## load crag_correction.yaml file
_count_queries=0

print("="*60)
print("Simulating Queries.................")
crag_query_file=load_faq(PROJECT_ROOT/'config'/'crag_correction.yaml')

for collection,question in crag_query_file.items():
    for q in question:
        _count_queries+=1

print(f"📋 Simulated {_count_queries} Queries in config/queries.yaml")
print("\n📝 Sample Queries:")
for collection,question in crag_query_file.items():
    print(f"   - {question[0]}")

print(f"   ... and {_count_queries - 8} more\n")
print("Queries Loaded")